In [1]:
#run this cell
    include("polynomials.jl")
    include("hypergeometricseries.jl")

rand (generic function with 74 methods)

# Making Conjectures on Hypergeometric series using a computer

## Introduction

This code contains three basic types of functions

- Polynomials

   The coefficients of the Polynomial can be from any sub-type of `Number`, and every polynomial is associated with an indeterminate (usually `X`).

In [2]:
Polynomial([0, 1], X)

1X + 0

In [3]:
x^3 - 5x + 1

1X^3 + -5X + 1

In [4]:
#Nested polynomials can be used for polynomials in several variables
(k + 3)*n^2 - (3k)*n + 1

(1K + 3)N^2 + (-3K + 0)N + (1)

- Hypergeometric terms

A hypergeometric term (a sequence $(c_n)_{n \in \mathbb{N}}$ where $c_0 = 1$ and $\frac{c_{k+1}}{c_k} = \frac{P(k)}{Q(k)}$, where $P$ and $Q$ are polynomials in $k$) is implemented as a `struct` containing the polynomials $P(k)$ and $Q(k)$ as data.

In [5]:
HypergeometricTerm(3k^2 - 1, 4k)

Hypergeometric term with ratio: 3K^2 + -1 // 4K + 0

In [6]:
HypergeometricTerm(3k^2 - 1, 4k)(2)

11//16

- Hypergeometric series

A hypergeometric series (the series $\sum_{k = 0}^{\infty} c_k$ corresponding to the hypergeometric term $(c_k)_{k \in \mathbb{N}}$) is also implemented as a struct containing two polynomials. In this code, by convention, the terms $c_k$ in a hypergeometric series are not just functions of $k$, but depend on another fixed parameter $n$.

Since infinite sums are difficult to handle on a computer, **the summation is actually taken from $k = 0$ to $n$**. This was chosen because most identities involving binomial coefficients are of this form.

In [10]:
HypergeometricSeries((3k^2 - 1)*n + (2k^0)*n^0, (4k)*n)

Summation of Hypergeometric term with ratio: (3K^2 + -1)N + (2) // (4K + 0)N + (0)

## The algorithm

The objective of the algorithm is to conjecture closed-form expressions (in the form of hypergeometric terms in `n`) for a hypergeometric series in `n` and `k`, being summed over `k`.

It accomplishes this by
- generating a random hypergeometric term or series $h$.
- evaluating it for the first $m$ values (usually $10$) of `n` (v = ($h(1), h(2), \ldots, h(m)$).
- storing this in a dictionary, with the key equal to the above set of points $v$ and the value being a list containing all matching hypergeometric series/terms with the same set of points.

Two non-trivially equivalent series or terms are checked for matches at more places, and then this match is stated as a conjecture.

(This is inspired by the algorithm behind the [Ramanujan machine](https://www.ramanujanmachine.com/).)

The actual code is given below:

In [8]:
#the number of iterations and the number of matches
n_iter, matches = 10_000, 10

(10000, 10)

In [9]:
#the main dictionary which stores all the series
dictionary = Dict{NTuple{matches, Rational}, Vector{SpecialFunction}}()

Dict{NTuple{10,Rational},Array{SpecialFunction,1}}()

In [17]:
#this cell can be executed multiple times to add more entries to the dictionary
for _ ∈ 1:n_iter
    try
        #picks a random hypergeometric term or series
        h = rand() < 0.5 ? rand(HypergeometricSeries{Int, K, N}) : rand(HypergeometricTerm{Int, N})
        #computes the list of values (at the first 10 places)
        values = ntuple(i -> h(i), matches)

        if haskey(dictionary, values)
            #checks whether there are no equivalent series already matched
            if !any([h.num * g.den == h.den * g.num for g ∈ dictionary[values] if typeof(g) == typeof(h)])
                #adds the series to the dictionary at the corresponding key
                push!(dictionary[values], h)
            end
        else
            #create a new array containing just `h`
            dictionary[values] = SpecialFunction[h]
        end
    catch error end
end

In [18]:
#filtering the matches
matched = Dict([(v, f) for (v, f) ∈ dictionary if length(f) > 1]);        

In [19]:
#displaying a particular set of hypergeometric terms/series that are conjectured to be equal
rand(matched).second
#this cell can be run multiple times to display different matches

2-element Array{SpecialFunction,1}:
                        Hypergeometric term with ratio: -1N + -1 // -1N + -2
 Summation of Hypergeometric term with ratio: (1K + -2)N + (0) // (1)N + (2)

## Advantages

- In this approach, conjectures can also be verified and disproved by a computer, using powerful established techniques such as the Wilf-Zeilberger algorithm and Gosper's algorithm. This automates the process of mathematics, in the specific case of binomial identities, from end-to-end.
- The Ramanujan machine has produced some very elegant conjectures about continued fractions, using the approach that the above algorithm is based on. One would expect to eventually find similar novel identities using this technique, if the above algorithm is run for a sufficiently long time.

## Disadvantages

- To ensure that matches are discovered within a reasonable amount of time, restrictions must be placed on the types of polynomials generated. In this code, for example, the polynomials are restricted to linear polynomials with coefficients in the range $\{-3, -2, -1, 0, 1, 2, 3\}$. This also limits the kinds of conjectures that are made. Also, in its present form, the machine can make conjectures involving just two variables - the regular variable `n` and the summation variable `k`.
- This is computationally very intensive, and requires a lot of time to generate novel results.

---